In [1]:
import os
import tables as tb

os.environ["POLARS_MAX_THREADS"] = "128"
import polars as pl
import polars.selectors as cs

In [2]:
%load_ext watermark
%watermark -vp polars,tables

Python implementation: CPython
Python version       : 3.11.3
IPython version      : 8.13.2

polars: 0.20.6
tables: 3.8.0



# Acquiring a candidate set of proteins for structural searches
To search for capsid folds, we used foldseek. Instead of directly predicting a structure (which can be computationally expensive), we used the ProstT5 model implemented in foldseek that translates amino acid alphabets (proteins) to a structural alphabet. This can use pre-existing text alignment strategies (notably implemented in mmseqs2, reimplemented for foldseek) to search structural alphabet databases instead of the most computationally expensive 3D structural alignment.

For our investigations, rather than focus on all proteins in our dataset (which would take a long time), we only kept proteins belonging to protein clusters that had either VOG-detectable capsids and unannotated proteins. Our implicit assumption is that these represent protein clusters that are structurally similar, which we indirectly infer by detecting if these unannotated proteins contain capsid-like structural folds.

In [3]:
# for simplicity we are changing the "vog_category"
# for major capsid proteins to "MCP" instead of "structural"
mcp_query = (
    pl.col("vog_annot").str.contains("(?i)(Major )?capsid protein") &
    (~pl.col("vog_annot").str.contains("Minor")) &
    pl.col("vog_bitscore").ge(75)
)

ptn_info = (
    pl.scan_csv("supplementary_table_2.tsv", separator="\t")
    .filter(pl.col("dataset") == "test")
    .with_columns(
        vog_category = (
            pl.when(mcp_query)
            .then(pl.lit("MCP"))
            .otherwise(pl.col("vog_category"))
        )
    )
    .drop(cs.starts_with("phrog"))
    .drop("genome", "genome_id", "dataset")
    .collect()
)



ptn_info.head()

ptn,ptn_id,vog_bitscore,vog_annot,vog_category
str,i64,f64,str,str
"""IMGVR_UViG_256…",0,null,"""unknown functi…","""unknown"""
"""IMGVR_UViG_256…",1,null,"""unknown functi…","""unknown"""
"""IMGVR_UViG_256…",2,null,"""unknown functi…","""unknown"""
"""IMGVR_UViG_256…",3,null,"""unknown functi…","""unknown"""
"""IMGVR_UViG_256…",4,null,"""unknown functi…","""unknown"""


We select our candidates from all combinations of protein-genome clustering. We only consider genome clustering with the hyperparameters of k=15 and resolution=1.0 (high):

In [4]:
candidates: list[pl.LazyFrame] = []

ptn_embedding_methods = ["esm-small", "esm-large", "pst-small", "pst-large", "genslm"]
with tb.open_file("embedding-based_protein_clusters_per_genome_cluster.h5") as fp:
    for gm_node in fp.root:
        genome_method = gm_node._v_name

        for ptn_method in ptn_embedding_methods:
            clustering_metadata = gm_node["metadata"][:]
            ptn_clusters = gm_node[ptn_method][:]
            genome_clusters = gm_node["genome"][:]

            for pclu, gclu, (genome_k, genome_res, protein_k, protein_res) in zip(
                ptn_clusters, genome_clusters, clustering_metadata
            ):
                if (genome_k != 15) or (genome_res != 1.0) or (protein_res == 0.85):
                    continue

                ptn_df = (
                    ptn_info
                    .lazy()
                    .with_columns(
                        genome_cluster = pl.lit(gclu),
                        protein_cluster = pl.lit(pclu),
                    )
                    .with_columns(
                        cluster_size = pl.len().over("genome_cluster", "protein_cluster")
                    )
                    .filter(pl.col("cluster_size") > 1)
                    .with_columns(
                        num_categories = pl.n_unique("vog_category").over("genome_cluster", "protein_cluster"),
                        interested = (
                            pl.col("vog_category")
                            .is_in(["MCP", "unknown"])
                            .all()
                            .over("genome_cluster", "protein_cluster")
                        )
                    )
                    .filter(
                        (pl.col("num_categories") == 2)
                        & pl.col("interested")
                    )
                    .select("ptn")
                )

                candidates.append(ptn_df)

candidate_proteins = (
    pl.concat(pl.collect_all(candidates))
    ["ptn"]
    .value_counts()
    .filter(pl.col("count") >= 10)
)

len(candidate_proteins)

72322

To this set of 72,322 proteins, we considered that these proteins belong to sequence identity-based clusters, since those are the most similar proteins in the traditional sense. These additional proteins are homologs to those above. Think of these more as like positive controls.

In [5]:
mmseqs_clusters = (
    pl.read_csv(
        "sequence_identity_clusters.tsv", 
        separator="\t",
        has_header=False,
        new_columns=["rep", "ptn"],
    )
    .with_columns(
        cluster_size = pl.col("ptn").len().over("rep"),
        cluster_id = pl.col("rep").rle_id(),
    )
)

First we need to detect the mmseqs clusters that contain capsids:

In [6]:
detectable_mcp_clusters = (
    ptn_info
    .filter(pl.col("vog_category") == "MCP")
    .select("ptn")
    .join(mmseqs_clusters, on="ptn")
    .filter(pl.col("cluster_size") > 1)
    ["cluster_id"]
)

len(detectable_mcp_clusters)

46564

Then choose all mmseqs clusters that contain both capsids and unannotated proteins:

In [7]:
unknown_with_mcp_clusters = (
    ptn_info
    .join(mmseqs_clusters, on="ptn")
    .filter(pl.col("cluster_id").is_in(detectable_mcp_clusters))
    .group_by("cluster_id")
    .agg(
        pl.first("cluster_size"),
        cats = pl.col("vog_category").unique(),
    )
    # should also have at least 1 MCP due to the filter on line 4
    .filter(pl.col("cats").list.contains("unknown"))
    ["cluster_id"]
)

mmseqs_ptn_candidates = (
    mmseqs_clusters
    .filter(pl.col("cluster_id").is_in(unknown_with_mcp_clusters))
    ["ptn"]
)

len(mmseqs_ptn_candidates)

40357

Thus, the total number of candidate proteins to search for structural similarity to capsid proteins is:

In [8]:
total_candidates = set(candidate_proteins["ptn"]) | set(mmseqs_ptn_candidates)

len(total_candidates)

100704

We then extracted the 100,704 candidated MCPs into a separate FASTA file and ran the following `foldseek` (v9.427df8a) commands to:
1. Convert these protein sequences in to the 3Di-structure sequence:

```bash
# download ProstT5 model
foldseek databases ProstT5 prostt5 tmp

# translate protein amino acids to structure
foldseek createdb FASTAFILE structDB --prostt5-model prostt5
```

2. Search against Protein Data Bank structures

```bash
# use foldseek to download structures
foldseek databases PDB pdb tmp

# then search
foldseek search structDB pdb/pdb searchDB tmp
```

3. Then compute alignment stats

```bash
foldseek convertalis structDB pdb/pdb searchDB
```

The structural alignments from this analysis are available with the supplementary data at the path: `fig4/capsid_candidates_structural_search_against_pdb.tsv`

Then, to determine which PDB structures corresponded to capsids, we queried PDB at the web server ([https://www.rcsb.org](https://www.rcsb.org)) with: `capsid, major capsid, coat, minor capsid, virion`, and selected all PDB IDs the came up. This list of capsid PDB IDs is available with supplementary data at: `fig4/capsid_pdb_ids.txt`.

# Analyzing structural searches

We used the above 2 files to process the structural alignment results:

In [9]:
foldseek_columns = "query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qlen,tlen".split(",")

with open("supplementary_data/fig4/capsid_pdb_ids.txt") as fp:
    capsid_pdb_ids = {line.rstrip() for line in fp}

struct_aln = (
    pl.scan_csv(
        "supplementary_data/fig4/capsid_candidates_structural_search_against_pdb.tsv",
        separator="\t",
        has_header=False,
        new_columns=foldseek_columns,
    )
    .filter(pl.col("bits") >= 100.)
    .with_columns(
        pdb_id = pl.col("target").str.extract("^(.*)?-.*$").str.to_uppercase(),
        chain = pl.col("target").str.split("_").list.get(1).str.to_uppercase(),
    )
    .with_columns(
        is_capsid = pl.col("pdb_id").is_in(capsid_pdb_ids),
    )
    .sort("bits", descending=True)
    # choose best PDB hit for each query protein
    .unique(subset="query", keep="first")
    .collect()
)

struct_aln

query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qlen,tlen,pdb_id,chain,is_capsid
str,str,f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,str,str,bool
"""IMGVR_UViG_GVM…","""8h2i-assembly1…",0.306,516,300,0,3,435,2,517,6.8630e-36,1581,437,517,"""8H2I""","""BU""",true
"""IMGVR_UViG_330…","""2fs3-assembly1…",0.196,287,206,0,155,441,1,257,3.1010e-14,630,448,258,"""2FS3""","""F""",true
"""IMGVR_UViG_GVM…","""8h2i-assembly1…",0.342,540,286,0,3,542,2,436,9.7660e-39,1650,542,436,"""8H2I-ASSEMBLY1…","""AB-5""",false
"""IMGVR_UViG_330…","""6ibc-assembly1…",0.166,365,296,0,2,357,10,374,8.0110e-13,563,358,374,"""6IBC-ASSEMBLY1…","""A-60""",false
"""IMGVR_UViG_330…","""1if0-assembly1…",0.134,299,219,0,94,392,3,256,2.4010e-9,431,393,256,"""1IF0""","""D""",true
"""IMGVR_UViG_275…","""2fs3-assembly1…",0.754,277,68,0,106,382,3,279,2.8700e-37,1510,388,280,"""2FS3-ASSEMBLY1…","""A-60""",false
"""IMGVR_UViG_261…","""2r7h-assembly1…",0.103,165,138,0,2,166,3,157,2.8800e-7,331,169,159,"""2R7H""","""B""",false
"""IMGVR_UViG_330…","""7lcc-assembly1…",0.196,118,85,0,16,122,361,478,0.004488,180,290,1369,"""7LCC""","""A""",false
"""IMGVR_UViG_330…","""1gff-assembly1…",0.177,282,216,0,17,280,3,284,0.0003871,212,424,417,"""1GFF""","""1""",true


Now we compute for each protein cluster only containing only unannotated proteins and VOG-detectable capsids the proportion of unannotated proteins that have structural similarity to capsids in the above table:

In [10]:
results: list[pl.LazyFrame] = []

ptn_embedding_methods = [
    "esm-small", "esm-large", "pst-small", "pst-large", "genslm"

]
with tb.open_file("embedding-based_protein_clusters_per_genome_cluster.h5") as fp:
    for gm_node in fp.root:
        genome_method = gm_node._v_name

        for ptn_method in ptn_embedding_methods:
            clustering_metadata = gm_node["metadata"][:]
            ptn_clusters = gm_node[ptn_method][:]
            genome_clusters = gm_node["genome"][:]

            for pclu, gclu, (genome_k, genome_res, protein_k, protein_res) in zip(
                ptn_clusters, genome_clusters, clustering_metadata
            ):
                if (genome_k != 15) or (genome_res != 1.0) or (protein_res == 0.85):
                    continue

                cluster_df = (
                    ptn_info
                    .lazy()
                    .with_columns(
                        genome_cluster = pl.lit(gclu),
                        protein_cluster = pl.lit(pclu),
                    )
                    .with_columns(
                        cluster_size = pl.len().over("genome_cluster", "protein_cluster")
                    )
                    .filter(pl.col("cluster_size") > 1)
                    .join(struct_aln.lazy(), left_on="ptn", right_on="query")
                    .with_columns(
                        ptn_method=pl.lit(ptn_method),
                        genome_method=pl.lit(genome_method),
                        genome_k=pl.lit(genome_k),
                        genome_r=pl.lit(genome_res),
                        ptn_r=pl.lit(protein_res),
                        ptn_k=pl.lit(protein_k),
                    )
                )

                results.append(cluster_df)

cluster_summary = pl.concat(pl.collect_all(results))
cluster_summary

ptn,ptn_id,vog_bitscore,vog_annot,vog_category,genome_cluster,protein_cluster,cluster_size,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qlen,tlen,pdb_id,chain,is_capsid,ptn_method,genome_method,genome_k,genome_r,ptn_r,ptn_k
str,i64,f64,str,str,u32,u32,u32,str,f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,str,str,bool,str,str,i32,f64,f64,i32
"""IMGVR_UViG_257…",43,null,"""unknown functi…","""unknown""",2,1,16,"""4dam-assembly2…",0.162,104,85,0,1,103,1,104,0.0001624,262,105,108,"""4DAM""","""H""",false,"""esm-small""","""ctx-avg-large""",15,1.0,0.1,5
"""IMGVR_UViG_264…",1012,260.42,"""sp|P15895|ORF4…","""unknown""",60,0,15,"""4v8p-assembly4…",0.097,115,103,0,14,128,33,147,0.2194,121,143,156,"""4V8P""","""GP""",false,"""esm-small""","""ctx-avg-large""",15,1.0,0.1,5
"""IMGVR_UViG_264…",1014,null,"""unknown functi…","""unknown""",60,2,13,"""3pvv-assembly2…",0.086,58,49,0,45,99,23,80,0.6603,109,150,95,"""3PVV""","""B""",false,"""esm-small""","""ctx-avg-large""",15,1.0,0.1,5
"""IMGVR_UViG_264…",1016,332.97,"""sp|P15893|ORF2…","""unknown""",60,3,7,"""6p8v-assembly1…",0.165,146,107,0,75,203,72,217,0.00001,267,224,302,"""6P8V""","""B""",false,"""esm-small""","""ctx-avg-large""",15,1.0,0.1,5
"""IMGVR_UViG_264…",1021,null,"""unknown functi…","""unknown""",60,1,18,"""3pvv-assembly2…",0.086,58,49,0,45,99,23,80,0.6603,109,150,95,"""3PVV""","""B""",false,"""esm-small""","""ctx-avg-large""",15,1.0,0.1,5
"""IMGVR_UViG_264…",1022,null,"""unknown functi…","""unknown""",60,1,18,"""3pvv-assembly2…",0.086,58,49,0,45,99,23,80,0.6603,109,150,95,"""3PVV""","""B""",false,"""esm-small""","""ctx-avg-large""",15,1.0,0.1,5
"""IMGVR_UViG_264…",1023,177.56,"""sp|P15892|ORF1…","""unknown""",60,1,18,"""8snb-assembly1…",0.109,146,118,0,68,213,3,135,0.1312,124,423,529,"""8SNB""","""7G""",false,"""esm-small""","""ctx-avg-large""",15,1.0,0.1,5
"""IMGVR_UViG_264…",1025,null,"""unknown functi…","""unknown""",60,0,15,"""4oq2-assembly1…",0.177,117,94,0,1,115,22,138,0.001093,183,115,297,"""4OQ2""","""A""",false,"""esm-small""","""ctx-avg-large""",15,1.0,0.1,5
"""IMGVR_UViG_265…",1096,null,"""unknown functi…","""unknown""",64,3,15,"""7u8r-assembly1…",0.12,166,118,0,4,169,13,147,0.698,113,186,150,"""7U8R""","""H""",false,"""esm-small""","""ctx-avg-large""",15,1.0,0.1,5


In [11]:
group_cols = ["genome_method", "ptn_method", "genome_k", "genome_r", "ptn_k", "ptn_r"]

# need this to compute prop of only unannotated proteins that are capsids
detected_capsids = (
    cluster_summary
    .filter(pl.col("vog_category") != "unknown")
    .group_by(*group_cols, "genome_cluster", "protein_cluster")
    .agg(
        known_mcps = pl.len()
    )
)

detected_capsids

genome_method,ptn_method,genome_k,genome_r,ptn_k,ptn_r,genome_cluster,protein_cluster,known_mcps
str,str,i32,f64,i32,f64,u32,u32,u32
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,480,0,1
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,855,3,2
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,1290,2,2
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,2058,1,2
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,455,3,3
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,2264,1,3
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,2418,0,1
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,2988,0,1
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,1058,0,2


In [12]:
per_cluster_proportion_summary = (
    cluster_summary
    .filter(
        pl.col("is_capsid") & (pl.col("vog_category") == "unknown")
    )
    .group_by(*group_cols, "genome_cluster", "protein_cluster")
    .agg(
        pl.first("cluster_size"),
        capsid_hits = pl.len(),
    )
    .join(detected_capsids, on=group_cols + ["genome_cluster", "protein_cluster"])
    .with_columns(
        prop = pl.col("capsid_hits") / (pl.col("cluster_size") - pl.col("known_mcps")),
        weight = pl.col("cluster_size") / pl.sum("cluster_size").over(group_cols)
    )
    .with_columns(
        weighted_prop = pl.col("prop") * pl.col("weight")
    )
)

per_cluster_proportion_summary

genome_method,ptn_method,genome_k,genome_r,ptn_k,ptn_r,genome_cluster,protein_cluster,cluster_size,capsid_hits,known_mcps,prop,weight,weighted_prop
str,str,i32,f64,i32,f64,u32,u32,u32,u32,u32,f64,f64,f64
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,3136,0,12,1,1,0.090909,0.000462,0.000042
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,3178,2,13,1,2,0.090909,0.0005,0.000045
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,2151,0,18,1,1,0.058824,0.000693,0.000041
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,9726,0,12,2,1,0.181818,0.000462,0.000084
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,2002,0,21,1,1,0.05,0.000808,0.00004
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,24268,0,10,1,2,0.125,0.000385,0.000048
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,19901,0,6,1,1,0.2,0.000231,0.000046
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,2169,0,20,1,1,0.052632,0.00077,0.000041
"""ctx-avg-large""","""esm-small""",15,1.0,5,0.1,26391,1,23,1,1,0.045455,0.000885,0.00004


Then, we use a weighted mean (weights = protein cluster size) to summarize for each genome clustering / protein clustering configuration:

In [13]:
proportion_summary = (
    per_cluster_proportion_summary
    .group_by(group_cols)
    .agg(
        pl.sum("weighted_prop", "cluster_size")
    )
    .rename({"weighted_prop": "prop"})
    .sort(["prop", "cluster_size"], descending=True)
)

proportion_summary

genome_method,ptn_method,genome_k,genome_r,ptn_k,ptn_r,prop,cluster_size
str,str,i32,f64,i32,f64,f64,u32
"""genslm""","""pst-small""",15,1.0,5,0.5,0.387778,3299
"""ctx-avg-large""","""pst-small""",15,1.0,5,0.5,0.376659,3062
"""esm2_t30_150M""","""pst-small""",15,1.0,5,0.5,0.374475,2921
"""pst-small""","""pst-small""",15,1.0,5,0.5,0.371701,2875
"""hyena-dna""","""pst-small""",15,1.0,5,0.5,0.370921,3202
"""kmer""","""genslm""",15,1.0,5,0.5,0.370719,4374
"""kmer""","""pst-small""",15,1.0,5,0.5,0.368512,2985
"""pst-large""","""pst-small""",15,1.0,5,0.5,0.36673,3173
"""esm2_t6_8M""","""pst-small""",15,1.0,5,0.5,0.363603,3146


The above table was used to make **Extended Data Figure 9A**.

For **Figure 4C**, we further summarized this to focus on the differences between protein embeddings used for clustering, regardless of genome clustering:

In [14]:
(
    proportion_summary
    .group_by("ptn_method", "ptn_k", "ptn_r")
    .agg(
        pl.mean("prop")
    )
    .sort(["ptn_k", "ptn_r", "prop"], descending=[False, False, True])
)

ptn_method,ptn_k,ptn_r,prop
str,i32,f64,f64
"""pst-small""",5,0.1,0.141046
"""pst-large""",5,0.1,0.105428
"""genslm""",5,0.1,0.084609
"""esm-small""",5,0.1,0.074845
"""esm-large""",5,0.1,0.073542
"""pst-small""",5,0.5,0.370149
"""genslm""",5,0.5,0.298805
"""pst-large""",5,0.5,0.23857
"""esm-small""",5,0.5,0.235549
